创建时间：20220628

更新时间：20220628

作者：Reeve 张小小

## 爬虫：通过编写程序来获取到互联网上的资源

百度

需求：用程序模拟浏览器，输入一个网址，从该网址中获取资源或者内容

In [1]:
from urllib.request import urlopen

In [2]:
url = 'http://www.baidu.com'
resp = urlopen(url)

# print(resp.read().decode("utf-8"))
# content="text/html;charset=utf-8"

with open('./mybaidu.html', mode='w') as f:
    f.write(resp.read().decode('utf-8'))   

resp.close() 

RMK: 同一个request不能read两遍，read一遍之后就会变成空白

In [3]:
print(resp.read().decode("utf-8"))

In [4]:
with open('./mybaidu.html', mode='w') as f:
    f.write(resp.read().decode('utf-8'))

## web请求过程解析

1. 服务器渲染：在服务器中直接把数据和html整合在一起，统一返回给浏览器（在页面源代码中看得到数据）

    e.g. 百度搜索周杰伦

2. 客户端渲染：第一次请求只给一个html骨架，第二次请求拿到数据，进行数据展示（在页面源代码中看不到数据）

    e.g. 豆瓣搜索憨豆

    所以关键在于找到第二次请求中的url，这需要我们熟练使用浏览器抓包工具（右键 检查）

## HTTP 协议

协议：就是两个计算机之间为了能够流畅的进行沟通而设置的一个君子协定。常见的协议有TCP/IP, SOAP协议, HTTP协议, SMTP协议等等...

HTTP协议，Hyper Text Transfer Protocol (超文本传输协议) 的缩写，适用于从万维网服务器传输超文本到本地浏览器的传送协议。直白点说，就是浏览器和服务器之间的数据交互遵守的协议

HTTP协议一般将一条信息分成三大块内容，无论是请求还是响应

请求：

```python
请求行 -> 请求方式（get/post/...） 请求url地址 协议
请求头 -> 放一些服务器要使用的附加信息

请求体 -> 一般放一些请求参数
```

响应：

```python
状态行 -> 协议 状态码
响应头 -> 放一些客户端要使用的附加信息

响应体 -> 服务器返回的真正客户端要使用的内容（HTML, json）等
```

RMK： 在爬虫的时候要格外注意请求头和响应头，这两个地方一般都隐含着一些比较重要的内容

请求头中最常见的一些内容（爬虫需要）：
1. User-Agent: 请求载体的身份标识（用什么浏览器、系统发送的请求）
2. Referer: 防盗链（这次请求是从哪个页面来的，反爬会用到）
3. cookie: 本地字符串数据信息（用户登录信息，反爬的token）

响应头的一些重要内容：
1. cookie: 本地字符串数据信息（用户登录信息，反爬的token）
2. 各种申请的字符串（需要经验，一般都是token字样，防止各种攻击和反爬）

请求方式：

GET: 显示提交

POST: 隐式提交

## Request 入门

In [5]:
import requests

In [6]:
url = 'https://www.sogou.com/web?query=周杰伦'


# 因为网页会识别user-agent，从而拦截
# 所以我们需要修改user-agent，进行伪装
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36 Edg/103.0.1264.37'
}

resp = requests.get(url, headers=headers)
print(resp)


<Response [200]>


In [7]:
print(resp.text)
resp.close()

<!DOCTYPE html><html data-vtype="default"> <head><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta http-equiv="x-dns-prefetch-control" content="on"><meta charset="utf-8"><meta name="referrer" content="always"><meta name="viewport" content="width=device-width,minimum-scale=1,maximum-scale=1,user-scalable=no"><meta name="format-detection" content="telephone=no">  <link rel="dns-prefetch" href="//img01.sogoucdn.com"><link rel="dns-prefetch" href="//img02.sogoucdn.com"><link rel="dns-prefetch" href="//img03.sogoucdn.com"><link rel="dns-prefetch" href="//img04.sogoucdn.com"><link rel="dns-prefetch" href="//dlweb.sogoucdn.com"><link rel="dns-prefetch" href="//search.sogoucdn.com"><link rel="preconnect" href="//img01.sogoucdn.com"><link rel="preconnect" href="//img02.sogoucdn.com"><link rel="preconnect" href="//img03.sogoucdn.com"><link rel="preconnect" href="//img04.sogoucdn.com"><link rel="preconnect" href="//dlweb.sogoucdn.com"><link rel="dns-prefetch" href="//search.sogoucdn.com"

POST请求中，输入参数不是整合在网页中的，所以我们需要另一种方法

In [8]:
url = 'https://fanyi.baidu.com/sug'

s = 'dog'
dat = {
    'kw': s
}

# 发送post请求
resp = requests.post(url, data=dat, headers=headers)
print(resp.json())
resp.close()

{'errno': 0, 'data': [{'k': 'dog', 'v': 'n. 狗; 蹩脚货; 丑女人; 卑鄙小人 v. 困扰; 跟踪'}, {'k': 'DOG', 'v': 'abbr. Data Output Gate 数据输出门'}, {'k': 'doge', 'v': 'n. 共和国总督'}, {'k': 'dogm', 'v': 'abbr. dogmatic 教条的; 独断的; dogmatism 教条主义; dogmatist'}, {'k': 'Dogo', 'v': '[地名] [马里、尼日尔、乍得] 多戈; [地名] [韩国] 道高'}]}


In [9]:
url = 'https://movie.douban.com/j/chart/top_list'

# 封装参数
param = {
    'type': '24', 
    'interval_id': '100:90', 
    'action': '', 
    'start': 0, 
    'limit': 20
}

resp = requests.get(url=url, params=param, headers=headers)
print(resp.text)
resp.close()

[{"rating":["9.6","50"],"rank":1,"cover_url":"https://img2.doubanio.com\/view\/photo\/s_ratio_poster\/public\/p2578474613.jpg","is_playable":true,"id":"1292063","types":["剧情","喜剧","爱情","战争"],"regions":["意大利"],"title":"美丽人生","url":"https:\/\/movie.douban.com\/subject\/1292063\/","release_date":"2020-01-03","actor_count":29,"vote_count":1220292,"score":"9.6","actors":["罗伯托·贝尼尼","尼可莱塔·布拉斯基","乔治·坎塔里尼","朱斯蒂诺·杜拉诺","赛尔乔·比尼·布斯特里克","玛丽萨·帕雷德斯","霍斯特·布赫霍尔茨","利迪娅·阿方西","朱利亚娜·洛约迪切","亚美利哥·丰塔尼","彼得·德·席尔瓦","弗朗西斯·古佐","拉法埃拉·莱博罗尼","克劳迪奥·阿方西","吉尔·巴罗尼","马西莫·比安奇","恩尼奥·孔萨尔维","吉安卡尔洛·科森蒂诺","阿伦·克雷格","汉尼斯·赫尔曼","弗兰科·梅斯科利尼","安东尼奥·普雷斯特","吉娜·诺维勒","理查德·塞梅尔","安德烈提多娜","迪尔克·范登贝格","奥梅罗·安东努蒂","沈晓谦","张欣"],"is_watched":false},{"rating":["9.6","50"],"rank":2,"cover_url":"https://img9.doubanio.com\/view\/photo\/s_ratio_poster\/public\/p2354179225.jpg","is_playable":false,"id":"5133063","types":["喜剧"],"regions":["英国"],"title":"憨豆先生精选辑","url":"https:\/\/movie.douban.com\/subject\/5133063\/","release_date":"1995-12-15","actor_coun